In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
import time
import tensorflow as tf
import numpy as np # linear algebra
import random
import os 
os.environ['PYTHONHASHSEED'] = '11'
np.random.seed(22)
random.seed(33)
tf.set_random_seed(44)

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
import gc

from keras import initializers, regularizers, constraints
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPooling1D, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from keras.callbacks import Callback
from keras.models import clone_model
import keras.backend as K

In [ ]:
max_features = 90000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 50
t0 = time.time()

In [ ]:
train_df = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
##test_df = pd.read_csv("../input/test.csv")
print("Train shape : ",train_df.shape)
##print("Test shape : ",test_df.shape)

Extratcing the stopwords in English language

In [ ]:
#from nltk import corpus
#stopwords=corpus.stopwords.words('english')

# fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
#test_X = test_df["question_text"].fillna("_na_").values


Stemming (result includes words not in stopword list)

In [ ]:
#from nltk.stem import PorterStemmer
#from nltk.tokenize import sent_tokenize, word_tokenize

train_X = train_df['question_text']
train_X = train_X.tolist()

qid_train = train_df['qid']
qid_train = qid_train.tolist()

##test_X = test_df['question_text']
##test_X = test_X.tolist()


Other Preprocesing : converting into lower case, extracting only alphabetical characters

Other preprocessing : Tokenization, Padding/Truncating questions

In [ ]:
# ## split to train and val
# train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)

# Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features,
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'’“”')


tokenizer.fit_on_texts(train_X)
train_X = tokenizer.texts_to_sequences(train_X)

# Pad the sentences 
trunc = 'pre'
train_X = pad_sequences(train_X, maxlen=maxlen, truncating=trunc)

##test_X = pad_sequences(test_X, maxlen=maxlen, truncating=trunc)

# Get the target values
train_y = train_df['target'].values

Splitting data into Train and Test

In [ ]:
test_X = train_X[1000000:]
train_X = train_X[:1000000]
test_y = train_y[1000000:]
train_y = train_y[:1000000]

Creating Glove Embedding

In [ ]:
EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_1 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_1[i] = embedding_vector

del embeddings_index; gc.collect() 

Creating Wiki News Embedding

In [ ]:
EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_2 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_2[i] = embedding_vector
        
del embeddings_index; gc.collect() 

Creating Paragram Embedding

In [ ]:
EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_3 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_3[i] = embedding_vector

del embeddings_index; gc.collect() 

Creating Gensim Embedding

In [ ]:
# https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
from gensim.models import KeyedVectors

EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_4 = (np.random.rand(nb_words, embed_size) - 0.5) / 5.0
for word, i in word_index.items():
    if i >= max_features: continue
    if word in embeddings_index:
        embedding_vector = embeddings_index.get_vector(word)
        embedding_matrix_4[i] = embedding_vector
        
del embeddings_index; gc.collect()

In [ ]:
del EMBEDDING_FILE
#del preprocessed_content1_train
#del qid_train, stemming_result_train
#del stopwords
del train_df

In [ ]:
gc.collect()

Calculating IDF

In [ ]:
from scipy.io import loadmat
idf = loadmat('../input/idf-mat-file/IDF.mat')

In [ ]:
word_extra_features=idf['word_extra_features']
vocab_word_freq=idf['vocab']
embed_size = 1
emb_mean, emb_std = word_extra_features.mean(), word_extra_features.std()
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_5 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    if word in vocab_word_freq:
        embedding_vector = word_extra_features[list(vocab_word_freq).index(word)]
        embedding_matrix_5[i] = embedding_vector

Concatinating the embeddings

In [ ]:
embedding_matrix = np.concatenate((embedding_matrix_1, embedding_matrix_2, embedding_matrix_3, embedding_matrix_4, embedding_matrix_5), axis=1)

del idf, embedding_matrix_1, embedding_matrix_2, embedding_matrix_3, embedding_matrix_4, embedding_matrix_5
gc.collect()
np.shape(embedding_matrix)

Utilities for Neural Network model

Attention Function

In [ ]:
from keras.engine.topology import Layer

class Attention(Layer):
    def __init__(self, step_dim=50,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

Creating the model

In [ ]:
from keras.layers import Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D

def create_rnn_model(rnn, lstm, maxlen, embedding, max_features, embed_size,
                     rnn_dim=64, dense1_dim=100, dense2_dim=50,
                     embed_trainable=False, seed=123):
    inp = Input(shape=(maxlen,), dtype='int32')
    x = Embedding(max_features, embed_size, weights=[embedding],
                  trainable=embed_trainable)(inp)
    #x = Conv1D(62,1)(x)
    #x = MaxPooling1D(2)(x)
    #x = Bidirectional(rnn(rnn_dim, return_sequences=True,
                          #kernel_initializer=glorot_uniform(seed=seed)))(x)
    x = Bidirectional(lstm(rnn_dim, kernel_initializer=glorot_uniform(seed=seed),
                           return_sequences=True))(x)
    
    
    x = Attention(maxlen)(x)
    
    #x = GlobalMaxPooling1D()(x)
    #x = Flatten()(x)
    merged = Dense(dense1_dim, activation='relu')(x)
    merged = BatchNormalization()(merged)
    preds = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[inp], \
        outputs=preds)
    model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])
    return model

In [ ]:
k = 4
num_val_samples = 200000 
num_epochs = 2
all_scores = []

In [ ]:
gc.collect()

Traaining the model with cross validation

In [ ]:
def f1_best(y_val, pred_val):
    return metrics.f1_score(y_val,pred_val)
    '''
    best_f1 = 0
    best_thresh = 0
    for thresh in np.linspace(0.2, 0.4, 41):
        f1 = metrics.f1_score(y_val, (pred_val > thresh).astype(int))
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh
    return best_f1, best_thresh
    '''

embed_ids = [list(range(300)), list(range(300, 600)),
             list(range(600, 900)), list(range(900, 1200))]
embed_ids_dict = {1: [embed_ids[0], embed_ids[1], embed_ids[2], embed_ids[3]],
                  2: [embed_ids[0] + embed_ids[1],
                      embed_ids[0] + embed_ids[2],
                      embed_ids[0] + embed_ids[3],
                      embed_ids[1] + embed_ids[2],
                      embed_ids[1] + embed_ids[3],
                      embed_ids[2] + embed_ids[3]],
                  3: [embed_ids[0] + embed_ids[1] + embed_ids[2],
                      embed_ids[0] + embed_ids[1] + embed_ids[3],
                      embed_ids[0] + embed_ids[2] + embed_ids[3],
                      embed_ids[1] + embed_ids[2] + embed_ids[3]],
                  4: [embed_ids[0] + embed_ids[1] + embed_ids[2] + embed_ids[3]]}
embed_ids_lst = embed_ids_dict[2]
embed_size = 1201

rnn = CuDNNGRU
lstm = CuDNNLSTM
embed_trainable = False

n_models = 1
#epochs = 7
batch_size = 128
dense1_dim = rnn_dim = 128
dense2_dim = 2 * rnn_dim

ema_n = int(len(train_y) / batch_size / 10)
decay = 0.9
scores = []
seed = 101 + 11 * 1
    
for i in range(n_models):
    t1 = time.time()
    seed = 101 + 11 * i
    cols_in_use = embed_ids_lst[i % len(embed_ids_lst)]
    
    model = create_rnn_model(rnn, lstm, maxlen, embedding_matrix,
                             max_features, embed_size,
                             rnn_dim=rnn_dim,
                             dense1_dim=dense1_dim,
                             dense2_dim=dense2_dim,
                             embed_trainable=embed_trainable,
                             seed=seed)
    model.summary()

    
    #cross_validation 
    for i in range(k):
        print(f'Processing fold # {i}')
        val_data = train_X[i * num_val_samples: (i+1) * num_val_samples]
        val_targets = train_y[i * num_val_samples: (i+1) * num_val_samples]
    
        partial_train_data = np.concatenate(
                                [train_X[:i * num_val_samples],
                                train_X[(i+1) * num_val_samples:]],
                                axis=0)
        partial_train_targets = np.concatenate(
                                [train_y[:i * num_val_samples],
                                train_y[(i+1)*num_val_samples:]],
                                axis=0)
        res = model.fit(x=partial_train_data,
                        y=partial_train_targets,
                        batch_size=batch_size,
                        epochs = num_epochs,
                        verbose=1,
                        validation_data = (val_data,val_targets))
        print("Scores: ", res)
    

In [ ]:
model.save("../working/model_fscore_nppreprocessing_attention&idf.h5")

Calculating F1 Score for the test data

In [ ]:
f1_avg = f1_best(test_y, np.round(model.predict(test_X)))
print(f' F1:{f1_avg:.4f}')

Plot the training result

In [ ]:
from matplotlib import pyplot as plt

def plot_history(Exp_history):
    plt.figure(1)
    plt.subplot(211)
    plt.plot(Exp_history.history['acc'])
    plt.plot(Exp_history.history['val_acc'])
    #plt.title('accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoches')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.subplot(212)
    plt.plot(Exp_history.history['loss'])
    plt.plot(Exp_history.history['val_loss'])
    #plt.title('loss')
    plt.ylabel('loss')
    plt.xlabel('epoches')
    plt.legend(['train', 'validation'], loc='upper left')
    #plt.savefig(results_path+'/train_history/Exp_train_history_1.png', bbox_inches='tight')
    plt.show()

In [ ]:
plot_history(res)